# 3 casos de uso

1. Docuemtación de codigo dado unos formatos
2. Pruebas funcionales
3. Conocimiento del Chapter Backend Satelites

# Chatbot con ChatterBot y spaCy

Este notebook muestra cómo adquirir un corpus con pandas, procesarlo con spaCy, entrenar un chatbot con ChatterBot y probarlo en CMD.

In [1]:
%pip install spacy transformers pandas torch googletrans==4.0.0-rc1 sqlite3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [3]:
import pandas as pd
import spacy
#from chatterbot import ChatBot
#from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
import sqlite3


## Adquisición del Corpus con Pandas

In [4]:
# Cargar los datasets
df1 = pd.read_csv('archive\\Ubuntu-dialogue-corpus\\dialogueText_196.csv')
df2 = pd.read_csv('archive\\Ubuntu-dialogue-corpus\\dialogueText_301.csv')

# Combinar los datasets
df = pd.concat([df1, df2])

# Asegurarse de que todos los valores en la columna 'text' sean cadenas de texto
df['text'] = df['text'].astype(str)

# Mostrar las primeras filas del dataset
df.head()

## Procesamiento del Texto con spaCy

In [5]:
nlp = spacy.load("en_core_web_sm")
# Preprocesar el texto con progreso
def preprocess_text(text, index, total):
    doc = nlp(text)
    if index % 1000000 == 0:  # Imprimir cada 100 textos procesados
        print(f"Procesando texto {index + 1} de {total}")
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

print("Preprocesando el texto...")
total_texts = len(df)
df['processed_text'] = [preprocess_text(text, i, total_texts) for i, text in enumerate(df['text'])]
print("Texto preprocesado.")

# Mostrar las primeras filas del dataset procesado
print("Primeras filas del dataset procesado:")
print(df.head())


c:\Users\luis5\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preprocesando el texto...
Procesando texto 1 de 25800707
Procesando texto 1000001 de 25800707
Procesando texto 2000001 de 25800707
Procesando texto 3000001 de 25800707
Procesando texto 4000001 de 25800707
Procesando texto 5000001 de 25800707
Procesando texto 6000001 de 25800707
Procesando texto 7000001 de 25800707


## Entrenamiento del Chatbot con ChatterBot

In [4]:
# Crear un chatbot con Chatterbot
chatbot = ChatBot(
    'OscarBot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        'chatterbot.logic.BestMatch',
        'chatterbot.logic.MathematicalEvaluation',
        'chatterbot.logic.TimeLogicAdapter'
    ],
    database_uri='sqlite:///database.sqlite3'
)

# Entrenar el chatbot con datos de conversación
trainer = ListTrainer(chatbot)

# Entrenar el chatbot con el texto procesado
for index, row in df.iterrows():
    trainer.train([row['processed_text']])

## Prueba del Chatbot en CMD

In [5]:
# Clase para manejar el contexto
class ContextManager:
    def __init__(self):
        self.context = []

    def update_context(self, user_input, bot_response):
        self.context.append({"user": user_input, "bot": bot_response})
        if len(self.context) > 10:  # Limitar el tamaño del contexto
            self.context.pop(0)

    def get_context(self):
        return self.context

# Crear una conexión a la base de datos
conn = sqlite3.connect('chatbot_memory.db')
c = conn.cursor()

# Crear una tabla para almacenar la memoria
c.execute('''CREATE TABLE IF NOT EXISTS memory
             (id INTEGER PRIMARY KEY, user_input TEXT, bot_response TEXT)''')

# Función para guardar en la memoria
def save_to_memory(user_input, bot_response):
    c.execute("INSERT INTO memory (user_input, bot_response) VALUES (?, ?)", (user_input, bot_response))
    conn.commit()

# Función para recuperar de la memoria
def retrieve_memory():
    c.execute("SELECT * FROM memory")
    return c.fetchall()

# Función para procesar la conversación
def process_conversation(user_input):
    processed_input = preprocess_text(user_input)
    response = chatbot.get_response(processed_input)
    context_manager.update_context(user_input, str(response))
    save_to_memory(user_input, str(response))
    return response

# Ejemplo de uso en CMD
if __name__ == "__main__":
    context_manager = ContextManager()
    
    print("OscarBot: Hola! ¿Cómo puedo ayudarte hoy?")
    
    while True:
        try:
            user_input = input("Tú: ")
            response = process_conversation(user_input)
            print(f"OscarBot: {response}")
        
        except(KeyboardInterrupt, EOFError, SystemExit):
            break